In [ ]:
import pandas as pd
import re
import os
from pathlib import Path
from datasets import Dataset, Audio, DatasetDict

In [ ]:
full_audio_segments = '/kaggle/input/full-audio-segments/segments'

In [ ]:
dataset = pd.read_csv('/kaggle/working/phonetcs_base_full.csv')
dataset_json = pd.read_json('/kaggle/input/output/output_cy_data.json')

In [ ]:
dataset.info()

In [ ]:
dataset = dataset.loc[:, ~dataset.columns.str.contains('^Unnamed')]
## Cleaning
dataset = dataset.drop_duplicates()
dataset = dataset.dropna()

In [ ]:
## Удаление пустых строк
dataset = dataset[dataset['text'].str.strip() != '']
dataset = dataset[dataset['pronunciation'].str.strip() != '']

In [ ]:
dataset.info()

In [ ]:
dataset = dataset[dataset['audio_segments'].apply(lambda x: Path(x).exists())]

In [ ]:
dataset.info()

In [ ]:
# Shuffle данных
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
dataset.info()

In [ ]:
dataset = dataset[['audio_segments', 'text', 'pronunciation']]

In [ ]:
dataset.info()

In [ ]:
hf_dataset = Dataset.from_pandas(dataset)

In [ ]:
hf_dataset = hf_dataset.rename_column("audio_segments", "audio")
hf_dataset = hf_dataset.rename_column("text", "sentence")

In [ ]:
hf_dataset = hf_dataset.cast_column("audio", Audio())

In [ ]:
hf_dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
hf_dataset

In [ ]:
from huggingface_hub import login
login()

In [ ]:
train_test = hf_dataset.train_test_split(test_size=0.2, seed=42)
test_val = train_test['test'].train_test_split(test_size=0.5, seed=42)

dataset_dict = DatasetDict({
   'train': train_test['train'],
   'validation': test_val['train'],
   'test': test_val['test']
})

In [ ]:
dataset_dict.push_to_hub("Elormiden/greek-speech-dataset")